In [5]:
import os
import pandas as pd 
import json
import traceback

In [31]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pandas

In [7]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
key=os.getenv('google_api_key')

In [10]:
pip install langchain-google-genai


Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [12]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.1,google_api_key=key)

In [13]:
llm.invoke("what is the best mcq movie avengers or spiderman").content.strip()


'There\'s no single "best" MCQ (multiple choice question) movie between Avengers and Spider-Man.  It depends entirely on what the questions are about.\n\n* **If the questions focus on overall cinematic scope and impact:** Avengers (specifically *Avengers: Endgame*) likely has a wider range of potential questions due to its larger cast, more complex plot, and greater impact on the MCU as a whole.\n\n* **If the questions focus on character development and relatable themes:** Spider-Man movies might offer more opportunities for questions about a single character\'s journey and struggles, making them potentially easier to create compelling MCQs around.\n\n* **If the questions focus on action sequences:** Both franchises have plenty of material, so it\'s a toss-up.\n\nTo make a good MCQ, you need to tailor the questions to the specific aspects of the movie you want to test.  Therefore, neither is inherently "better" for MCQs.'

In [14]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.1, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002255CD90850>, default_metadata=(), model_kwargs={})

In [15]:
pip install --upgrade langchain langchain-core langchain-google-genai


Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install langchain.chains

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchain.chains (from versions: none)
ERROR: No matching distribution found for langchain.chains


In [19]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
import PyPDF2

In [20]:
TEMPLATE='''
TEXT:{text}
you are an expert MCQ maker . Given the above text , it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in{tone} tone.
make sure that no question is repeated and check all the questions to be conforming the text as well
make sure to format your response like RESPONSE_JSON below and use it like a guide. \
Ensure that to make {number} MCQs
### RESPONSE_JSON
{response_json}

'''


In [21]:
prompt_template=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [22]:
chain=LLMChain(llm=llm,prompt=prompt_template,output_key="quiz",verbose=False)

C:\Users\thouf\AppData\Local\Temp\ipykernel_42820\3679699836.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain=LLMChain(llm=llm,prompt=prompt_template,output_key="quiz",verbose=False)


In [23]:
TEMPLATE2='''
you are an expert in english grammar and writer . Given a multiple choice quiz for {subject} students.\
you need to evaluate the complexity of the quiz and goive a complete analysis of the quiz . Only use at max 50 wordsfor complexity
if the quiz is not as per the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the studentsabilities
Quiz_MCQs:
{quiz}

check from an expert English Writer of the above quiz:
'''


In [24]:
quiz_evaluation_template=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [25]:
quiz_evaluation_chain=LLMChain(llm=llm,prompt=quiz_evaluation_template,output_key="review",verbose=False)

In [26]:
connection_chain=SequentialChain(chains=[chain,quiz_evaluation_chain],input_variables=["text","number","subject","tone","response_json"],output_variables=["quiz","review"],verbose=False)

In [27]:
file_path=r"C:\Users\thouf\mcqgeneratornew\data.txt"

In [28]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [29]:
TEXT

'In machine learning, deep learning focuses on utilizing multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be supervised, semi-supervised or unsupervised.[2]\n\nSome common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and b

In [ ]:
def estimate_gemini_token_usage(prompt_inputs: dict, model_outputs: dict = None) -> int:
    input_tokens = 0
    for key, value in prompt_inputs.items():
        if isinstance(value, str):
            input_tokens += len(value.split())
        elif isinstance(value, dict):
            input_tokens += len(json.dumps(value).split())
        else:
            input_tokens += len(str(value).split())
    output_tokens = 0
    if model_outputs:
        for key, value in model_outputs.items():
            if isinstance(value, str):
                
                output_tokens += len(value.split())
            elif isinstance(value, dict):
                output_tokens += len(json.dumps(value).split())
            else:
                output_tokens += len(str(value).split())
    return input_tokens + output_tokens
if __name__ == "__main__":
    inputs = {
        "text": TEXT,
        "number": 3,
        "subject": "Machine learning",
        "tone": "simple",
        "response_json": json.dumps(RESPONSE_JSON)
    }
    try:
        print("\nStarting the MCQ generation and evaluation process...")
        response = connection_chain.invoke(inputs)
        total_tokens = estimate_gemini_token_usage(inputs, response)
        print(f"Estimated Total Tokens Used: {total_tokens}")
        print("\nGenerated Quiz (Raw LLM Output) ")
        quiz_output = response.get("quiz", "No quiz generated or output is not a string.")
  
        print(f"\nAn error occurred during chain invocation: {e}")
        traceback.print_exc()       
        try:
            parsed_quiz = json.loads(quiz_output)
            print(json.dumps(parsed_quiz, indent=2))
        except json.JSONDecodeError:
            print("Quiz output is not valid JSON. Raw output:")
    except Exception as e:    
        print(quiz_output)
        print("\n Quiz Review")
        print(response.get("review", "No review generated."))
   


NameError: name 'RESPONSE_JSON' is not defined

In [ ]:
print(json.dumps(parsed_quiz, indent=2))

In [ ]:
print(quiz_table_data)

In [ ]:
import pandas as pd 

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%y_%H_%M_%S')